In [1]:
import pandas as pd
import json
import os

In [2]:
DELAYS = ['CARRIER_DELAY', 'WEATHER_DELAY', 'NAS_DELAY', 'SECURITY_DELAY', 'LATE_AIRCRAFT_DELAY']
DATES = ['YEAR', 'MONTH', 'DAY_OF_MONTH']
AIRLINE = 'AIRLINE_NAME'
ORIGIN = ['ORIGIN_AIRPORT_NAME', 'ORIGIN_STATE_ABR', 'ORIGIN_STATE_NM']
DEST = ['DEST_AIRPORT_NAME', 'DEST_STATE_ABR', 'DEST_STATE_NM']
DEPART = ['CRS_DEP_TIME', 'DEP_TIME', 'DEP_DELAY_NEW', 'TAXI_OUT', 'WHEELS_OFF']
ARRIVAL = ['CRS_ARR_TIME', 'ARR_TIME', 'ARR_DELAY_NEW', 'TAXI_IN', 'WHEELS_ON']
FLIGHT = ['CRS_ELAPSED_TIME', 'ACTUAL_ELAPSED_TIME', 'AIR_TIME', 'DISTANCE']

In [3]:
df = pd.read_csv('/Users/ykamoji/Documents/Semester4/COMPSCI_571/Project/Data/airlines_delay_data_v3.csv')

## Sanity Check

In [4]:
df.head(5)

,YEAR,MONTH,DAY_OF_MONTH,ORIGIN_STATE_ABR,AIRLINE_NAME,ORIGIN_AIRPORT_NAME,ORIGIN_STATE_NM,DEST_STATE_ABR,DEST_AIRPORT_NAME,DEST_STATE_NM,...,WHEELS_ON,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,DISTANCE,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY
0,2023,1,1,LA,Endeavor Air Inc.,"Alexandria, LA: Alexandria International",Louisiana,GA,"Atlanta, GA: Hartsfield-Jackson Atlanta Intern...",Georgia,...,2132,103.0,105.0,73.0,500.0,0,83,2,0,342
1,2023,1,1,LA,Endeavor Air Inc.,"Alexandria, LA: Alexandria International",Louisiana,GA,"Atlanta, GA: Hartsfield-Jackson Atlanta Intern...",Georgia,...,2144,109.0,90.0,75.0,500.0,11,0,0,0,61
2,2023,1,1,NY,Endeavor Air Inc.,"Albany, NY: Albany International",New York,MI,"Detroit, MI: Detroit Metro Wayne County",Michigan,...,1233,130.0,111.0,82.0,489.0,0,0,0,0,288
3,2023,1,1,GA,Endeavor Air Inc.,"Atlanta, GA: Hartsfield-Jackson Atlanta Intern...",Georgia,LA,"Alexandria, LA: Alexandria International",Louisiana,...,1821,108.0,115.0,82.0,500.0,61,0,7,0,9
4,2023,1,1,GA,Endeavor Air Inc.,"Atlanta, GA: Hartsfield-Jackson Atlanta Intern...",Georgia,GA,"Brunswick, GA: Brunswick Golden Isles",Georgia,...,1526,72.0,53.0,41.0,238.0,12,0,0,0,305


In [5]:
df[DELAYS].isnull().sum()

CARRIER_DELAY          0
WEATHER_DELAY          0
NAS_DELAY              0
SECURITY_DELAY         0
LATE_AIRCRAFT_DELAY    0
dtype: int64

In [6]:
df[DATES].isnull().sum()

YEAR            0
MONTH           0
DAY_OF_MONTH    0
dtype: int64

In [7]:
df[AIRLINE].isnull().sum()

0

In [8]:
df[ORIGIN].isnull().sum()

ORIGIN_AIRPORT_NAME    0
ORIGIN_STATE_ABR       0
ORIGIN_STATE_NM        0
dtype: int64

In [9]:
df[DEST].isnull().sum()

DEST_AIRPORT_NAME    0
DEST_STATE_ABR       0
DEST_STATE_NM        0
dtype: int64

In [10]:
df[DEPART].isnull().sum()

CRS_DEP_TIME     0
DEP_TIME         0
DEP_DELAY_NEW    0
TAXI_OUT         0
WHEELS_OFF       0
dtype: int64

In [11]:
df[ARRIVAL].isnull().sum()

CRS_ARR_TIME     0
ARR_TIME         0
ARR_DELAY_NEW    0
TAXI_IN          0
WHEELS_ON        0
dtype: int64

In [12]:
df[FLIGHT].isnull().sum()

CRS_ELAPSED_TIME       0
ACTUAL_ELAPSED_TIME    0
AIR_TIME               0
DISTANCE               0
dtype: int64

### Geo map calculations

In [13]:
df[['ORIGIN_STATE_ABR']+DELAYS].groupby('ORIGIN_STATE_ABR')[DELAYS].agg(['sum','count']).head(5)

CARRIER_DELAY         WEATHER_DELAY         NAS_DELAY  \
                           sum   count           sum   count       sum   
ORIGIN_STATE_ABR                                                         
AK                      283961   14356         50510   14356    124783   
AL                      392531   13718         90903   13718    155826   
AR                      304114   11272         99123   11272    148418   
AZ                     1924024   79167        154790   79167    713789   
CA                     6189688  281108        611038  281108   3400070   

                         SECURITY_DELAY         LATE_AIRCRAFT_DELAY          
                   count            sum   count                 sum   count  
ORIGIN_STATE_ABR                                                             
AK                 14356          10570   14356              374715   14356  
AL                 13718           1052   13718              470458   13718  
AR                 11272           1727   11272              366054   11272  
AZ                 79167           8302   79167             2272940   79167  
CA                281108          29019  281108             7956548  281108

In [14]:
df[['DEST_STATE_ABR'] + DELAYS].groupby('DEST_STATE_ABR')[DELAYS].agg(['sum', 'count']).head(10)

CARRIER_DELAY         WEATHER_DELAY         NAS_DELAY          \
                         sum   count           sum   count       sum   count   
DEST_STATE_ABR                                                                 
AK                    268383   16436         54825   16436    144806   16436   
AL                    373042   15003         89358   15003    138439   15003   
AR                    269449   12182         66390   12182    121457   12182   
AZ                   2319925   82990        267579   82990    686222   82990   
CA                   6760320  315847        838185  315847   3833055  315847   
CO                   3311156  136809        624845  136809   1962095  136809   
CT                    289375   10666         27621   10666    100410   10666   
FL                   7034324  298365        866917  298365   5216231  298365   
GA                   3901021  125704        561776  125704   1559574  125704   
HI                   1035271   45140         51226   45140    311022   45140   

               SECURITY_DELAY         LATE_AIRCRAFT_DELAY          
                          sum   count                 sum   count  
DEST_STATE_ABR                                                     
AK                      10510   16436              358609   16436  
AL                       1272   15003              406546   15003  
AR                       1033   12182              341045   12182  
AZ                      13538   82990             2620109   82990  
CA                      39061  315847             7804683  315847  
CO                      10951  136809             3843391  136809  
CT                       1272   10666              303578   10666  
FL                      46203  298365             8720495  298365  
GA                      15134  125704             3508415  125704  
HI                       5812   45140              826336   45140

In [15]:
df[df['CARRIER_DELAY'] == df['CARRIER_DELAY'].max()]

,YEAR,MONTH,DAY_OF_MONTH,ORIGIN_STATE_ABR,AIRLINE_NAME,ORIGIN_AIRPORT_NAME,ORIGIN_STATE_NM,DEST_STATE_ABR,DEST_AIRPORT_NAME,DEST_STATE_NM,...,WHEELS_ON,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,DISTANCE,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY
631403,2023,6,5,VA,Republic Airline,"Washington, DC: Ronald Reagan Washington National",Virginia,FL,"Jacksonville, FL: Jacksonville International",Florida,...,1411,124.0,140.0,112.0,634.0,5764,0,16,0,0


In [16]:
df.loc[df.groupby('ORIGIN_STATE_ABR')['LATE_AIRCRAFT_DELAY'].idxmax()].head(5)

,YEAR,MONTH,DAY_OF_MONTH,ORIGIN_STATE_ABR,AIRLINE_NAME,ORIGIN_AIRPORT_NAME,ORIGIN_STATE_NM,DEST_STATE_ABR,DEST_AIRPORT_NAME,DEST_STATE_NM,...,WHEELS_ON,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,DISTANCE,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY
876264,2023,7,20,AK,American Airlines Inc.,"Anchorage, AK: Ted Stevens Anchorage Internati...",Alaska,IL,"Chicago, IL: Chicago O'Hare International",Illinois,...,427,369.0,353.0,333.0,2846.0,0,0,0,0,1274
2972749,2024,12,28,AL,American Airlines Inc.,"Birmingham, AL: Birmingham-Shuttlesworth Inter...",Alabama,NC,"Charlotte, NC: Charlotte Douglas International",North Carolina,...,113,93.0,82.0,56.0,350.0,0,0,0,0,2354
1171760,2023,9,25,AR,SkyWest Airlines Inc.,"Texarkana, AR: Texarkana Regional-Webb Field",Arkansas,TX,"Dallas/Fort Worth, TX: Dallas/Fort Worth Inter...",Texas,...,2305,71.0,53.0,33.0,181.0,10,0,0,0,1761
829434,2023,7,12,AZ,American Airlines Inc.,"Phoenix, AZ: Phoenix Sky Harbor International",Arizona,NC,"Charlotte, NC: Charlotte Douglas International",North Carolina,...,341,253.0,226.0,206.0,1773.0,0,0,0,0,1833
1789023,2024,3,26,CA,American Airlines Inc.,"Los Angeles, CA: Los Angeles International",California,TX,"Dallas/Fort Worth, TX: Dallas/Fort Worth Inter...",Texas,...,201,178.0,165.0,141.0,1235.0,121,0,0,0,2506


In [17]:
df.loc[df.groupby('DEST_STATE_ABR')['CARRIER_DELAY'].idxmax()].head(5)

,YEAR,MONTH,DAY_OF_MONTH,ORIGIN_STATE_ABR,AIRLINE_NAME,ORIGIN_AIRPORT_NAME,ORIGIN_STATE_NM,DEST_STATE_ABR,DEST_AIRPORT_NAME,DEST_STATE_NM,...,WHEELS_ON,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,DISTANCE,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY
732208,2023,6,24,NJ,United Air Lines Inc.,"Newark, NJ: Newark Liberty International",New Jersey,AK,"Anchorage, AK: Ted Stevens Anchorage Internati...",Alaska,...,1508,463.0,459.0,416.0,3370.0,1230,0,0,0,0
706451,2023,6,20,TX,PSA Airlines Inc.,"Dallas/Fort Worth, TX: Dallas/Fort Worth Inter...",Texas,AL,"Huntsville, AL: Huntsville International-Carl ...",Alabama,...,912,115.0,135.0,86.0,603.0,1089,0,20,0,0
1864691,2024,4,12,CO,Mesa Airlines Inc.,"Denver, CO: Denver International",Colorado,AR,"Little Rock, AR: Bill and Hillary Clinton Nat ...",Arkansas,...,1036,128.0,119.0,94.0,771.0,1089,0,0,0,0
1841887,2024,4,8,CA,American Airlines Inc.,"San Diego, CA: San Diego International",California,AZ,"Phoenix, AZ: Phoenix Sky Harbor International",Arizona,...,2151,84.0,90.0,56.0,304.0,3300,0,6,0,48
82663,2023,1,21,IL,American Airlines Inc.,"Chicago, IL: Chicago O'Hare International",Illinois,CA,"Los Angeles, CA: Los Angeles International",California,...,2300,292.0,240.0,213.0,1744.0,2078,0,0,0,0


In [18]:
print(set(df['ORIGIN_STATE_NM'].unique()) == set(df['DEST_STATE_NM'].unique()))
# df['ORIGIN_STATE_NM'].unique()

True


In [19]:
states = set(["Alabama", "Alaska", "Arizona", "Arkansas", "California", "Colorado", "Connecticut", "Delaware",
    "Florida", "Georgia", "Hawaii", "Idaho", "Illinois", "Indiana", "Iowa", "Kansas", "Kentucky", "Louisiana",
    "Maine", "Maryland", "Massachusetts", "Michigan", "Minnesota", "Mississippi", "Missouri", "Montana",
    "Nebraska", "Nevada", "New Hampshire", "New Jersey", "New Mexico", "New York", "North Carolina",
    "North Dakota", "Ohio", "Oklahoma", "Oregon", "Pennsylvania", "Rhode Island", "South Carolina",
    "South Dakota", "Tennessee", "Texas", "Utah", "Vermont", "Virginia", "Washington", "West Virginia",
    "Wisconsin", "Wyoming"])

In [20]:
data_states = set(df['ORIGIN_STATE_NM'].unique())
print(states.difference(data_states))
print(data_states.difference(states))
# data_states

{'Delaware'}
{'U.S. Pacific Trust Territories and Possessions', 'U.S. Virgin Islands', 'Puerto Rico'}


In [30]:
origin_airports = set(df['ORIGIN_AIRPORT_NAME'].unique())
# origin_airports
# dest_airports = set(df['DEST_AIRPORT_NAME'].unique())
# print(origin_airports.difference(dest_airports))
# print(dest_airports.difference(origin_airports))

In [32]:
airlines = set(df[AIRLINE].unique())
# airlines

In [ ]:
print((df['ORIGIN_AIRPORT_NAME'].str.split(':', expand=True)[0] != df['ORIGIN_CITY_NAME']).sum())
print((df['DEST_AIRPORT_NAME'].str.split(':', expand=True)[0] != df['DEST_CITY_NAME']).sum())

In [21]:
print((df['ORIGIN_AIRPORT_NAME'].str.split(':', expand=True)[0].str.split(', ', expand=True)[1] != df['ORIGIN_STATE_ABR']).sum())
print((df['DEST_AIRPORT_NAME'].str.split(':', expand=True)[0].str.split(', ', expand=True)[1] != df['DEST_STATE_ABR']).sum())

96251
97968


In [22]:
df = df[df['CRS_ELAPSED_TIME'] >=0]
df['DATE'] = pd.to_datetime(df[DATES].astype(str).agg('-'.join, axis=1))
df['IS_WEEKDAY'] = pd.to_datetime(df['DATE']).dt.weekday < 5
df['YR-MN'] = df[['YEAR','MONTH']].astype(str).agg('-'.join, axis=1)
# df

In [23]:
df['congestion'] = (df['TAXI_OUT'] + df['TAXI_IN']) / df['CRS_ELAPSED_TIME']
df['congestion'].describe()

count    2.995619e+06
mean     2.663528e-01
std      1.955696e-01
min      7.957560e-03
25%      1.395349e-01
50%      2.096774e-01
75%      3.285714e-01
max      2.100000e+01
Name: congestion, dtype: float64

In [24]:
df['time_deviation'] = (df['ACTUAL_ELAPSED_TIME'] - df['CRS_ELAPSED_TIME'])/df['AIR_TIME'] 
df['time_deviation'].describe()

count    2.995619e+06
mean     3.652806e-02
std      2.651453e-01
min     -2.752000e+01
25%     -9.909910e-02
50%     -3.584229e-03
75%      1.196581e-01
max      1.400000e+01
Name: time_deviation, dtype: float64

In [25]:
df['crs_speed'] = df['DISTANCE']  / df['CRS_ELAPSED_TIME']
df['actual_speed'] = df['DISTANCE'] / df['ACTUAL_ELAPSED_TIME']
df['speed_deviation'] = (df['actual_speed'] - df['crs_speed']) / df['crs_speed']
df['speed_deviation'].describe()

count    2.995619e+06
mean    -9.009440e-03
std      1.418401e-01
min     -9.827586e-01
25%     -8.959538e-02
50%      3.257329e-03
75%      7.964602e-02
max      1.610204e+01
Name: speed_deviation, dtype: float64

In [26]:
df['dep_delay_m'] = df['DEP_DELAY_NEW']
df.loc[df['dep_delay_m'] == 0, 'dep_delay_m'] = df['ACTUAL_ELAPSED_TIME'] - df['CRS_ELAPSED_TIME']
df['recovery_efficiency'] = (df['ACTUAL_ELAPSED_TIME'] - df['CRS_ELAPSED_TIME']) / df['dep_delay_m']
df['recovery_efficiency'].describe()

count    2.995616e+06
mean     6.311012e-01
std      3.170872e+00
min     -2.100000e+01
25%     -1.538462e-01
50%     -3.098373e-03
75%      5.294118e-01
max      2.150000e+02
Name: recovery_efficiency, dtype: float64

In [27]:
calc_origin_df = df[['YR-MN', 'ORIGIN_STATE_ABR','ORIGIN_AIRPORT_NAME','AIRLINE_NAME', 'IS_WEEKDAY']+['congestion', 'time_deviation','speed_deviation','recovery_efficiency']]
calc_origin_df.columns = list(map(str, range(9)))
calc_origin_df = calc_origin_df.groupby(list(map(str, range(5))))[list(map(str, range(5, 9)))].agg(['mean'])
calc_origin_df.columns = calc_origin_df.columns.get_level_values(0)
calc_origin_df

5  \
0      1  2                                                  3                      4                 
2023-1 AK Adak Island, AK: Adak                              Alaska Airlines Inc.   False  0.115584   
          Anchorage, AK: Ted Stevens Anchorage International Alaska Airlines Inc.   False  0.197400   
                                                                                    True   0.183852   
                                                             American Airlines Inc. True   0.111392   
                                                             Delta Air Lines Inc.   False  0.152717   
...                                                                                             ...   
2024-9 WY Riverton/Lander, WY: Central Wyoming Regional      SkyWest Airlines Inc.  True   0.442222   
          Rock Springs, WY: Southwest Wyoming Regional       SkyWest Airlines Inc.  False  0.444444   
                                                                                    True   0.430556   
          Sheridan, WY: Sheridan County                      SkyWest Airlines Inc.  False  0.375000   
                                                                                    True   0.329545   

                                                                                                  6  \
0      1  2                                                  3                      4                 
2023-1 AK Adak Island, AK: Adak                              Alaska Airlines Inc.   False  0.043436   
          Anchorage, AK: Ted Stevens Anchorage International Alaska Airlines Inc.   False  0.087104   
                                                                                    True   0.080514   
                                                             American Airlines Inc. True   0.066489   
                                                             Delta Air Lines Inc.   False  0.041582   
...                                                                                             ...   
2024-9 WY Riverton/Lander, WY: Central Wyoming Regional      SkyWest Airlines Inc.  True   0.182804   
          Rock Springs, WY: Southwest Wyoming Regional       SkyWest Airlines Inc.  False  0.069767   
                                                                                    True   0.079018   
          Sheridan, WY: Sheridan County                      SkyWest Airlines Inc.  False  0.098631   
                                                                                    True   0.025039   

                                                                                                  7  \
0      1  2                                                  3                      4                 
2023-1 AK Adak Island, AK: Adak                              Alaska Airlines Inc.   False -0.036366   
          Anchorage, AK: Ted Stevens Anchorage International Alaska Airlines Inc.   False -0.066561   
                                                                                    True  -0.064808   
                                                             American Airlines Inc. True  -0.059524   
                                                             Delta Air Lines Inc.   False -0.033095   
...                                                                                             ...   
2024-9 WY Riverton/Lander, WY: Central Wyoming Regional      SkyWest Airlines Inc.  True  -0.103294   
          Rock Springs, WY: Southwest Wyoming Regional       SkyWest Airlines Inc.  False -0.040000   
                                                                                    True  -0.035855   
          Sheridan, WY: Sheridan County                      SkyWest Airlines Inc.  False -0.027512   
                                                                                    True  -0.008123   

                                                                              

In [28]:
condensed_origin_df = df[['YR-MN', 'ORIGIN_STATE_ABR','ORIGIN_AIRPORT_NAME','AIRLINE_NAME', 'IS_WEEKDAY']+DELAYS]
condensed_origin_df.columns = list(map(str, range(10)))
condensed_origin_df = condensed_origin_df.groupby(list(map(str, range(5))))[list(map(str, range(5, 10)))].agg(['sum'])
condensed_origin_df.columns = condensed_origin_df.columns.get_level_values(0)
condensed_origin_df

5  \
0      1  2                                                  3                      4             
2023-1 AK Adak Island, AK: Adak                              Alaska Airlines Inc.   False     0   
          Anchorage, AK: Ted Stevens Anchorage International Alaska Airlines Inc.   False   415   
                                                                                    True   1883   
                                                             American Airlines Inc. True    494   
                                                             Delta Air Lines Inc.   False   367   
...                                                                                         ...   
2024-9 WY Riverton/Lander, WY: Central Wyoming Regional      SkyWest Airlines Inc.  True    376   
          Rock Springs, WY: Southwest Wyoming Regional       SkyWest Airlines Inc.  False    77   
                                                                                    True    238   
          Sheridan, WY: Sheridan County                      SkyWest Airlines Inc.  False   426   
                                                                                    True    403   

                                                                                            6  \
0      1  2                                                  3                      4           
2023-1 AK Adak Island, AK: Adak                              Alaska Airlines Inc.   False   0   
          Anchorage, AK: Ted Stevens Anchorage International Alaska Airlines Inc.   False   0   
                                                                                    True   98   
                                                             American Airlines Inc. True    0   
                                                             Delta Air Lines Inc.   False  71   
...                                                                                        ..   
2024-9 WY Riverton/Lander, WY: Central Wyoming Regional      SkyWest Airlines Inc.  True    5   
          Rock Springs, WY: Southwest Wyoming Regional       SkyWest Airlines Inc.  False   0   
                                                                                    True    0   
          Sheridan, WY: Sheridan County                      SkyWest Airlines Inc.  False   0   
                                                                                    True    0   

                                                                                              7  \
0      1  2                                                  3                      4             
2023-1 AK Adak Island, AK: Adak                              Alaska Airlines Inc.   False    25   
          Anchorage, AK: Ted Stevens Anchorage International Alaska Airlines Inc.   False   781   
                                                                                    True   2131   
                                                             American Airlines Inc. True     25   
                                                             Delta Air Lines Inc.   False   133   
...                                                                                         ...   
2024-9 WY Riverton/Lander, WY: Central Wyoming Regional      SkyWest Airlines Inc.  True     59   
          Rock Springs, WY: Southwest Wyoming Regional       SkyWest Airlines Inc.  False     3   
                                                                                    True     26   
          Sheridan, WY: Sheridan County                      SkyWest Airlines Inc.  False     0   
                                                                                    True     17   

                                                                                            8  \
0      1  2                                                  3                      4           
2023-1 AK Adak Island, AK: Adak                              A

In [14]:
# df[(df['ORIGIN_STATE_ABR'] == 'AK') & (df['ORIGIN_AIRPORT_NAME'] == 'Anchorage, AK: Ted Stevens Anchorage International') & (df['AIRLINE_NAME'] == 'Delta Air Lines Inc.') & (df['MONTH'] == 1) & (df['IS_WEEKDAY'] == False) & (df['YEAR'] == 2023)]

In [29]:
condensed_origin_df[list(map(str, range(10, 14)))] = calc_origin_df[list(map(str, range(5, 9)))]
condensed_origin_df

5  \
0      1  2                                                  3                      4             
2023-1 AK Adak Island, AK: Adak                              Alaska Airlines Inc.   False     0   
          Anchorage, AK: Ted Stevens Anchorage International Alaska Airlines Inc.   False   415   
                                                                                    True   1883   
                                                             American Airlines Inc. True    494   
                                                             Delta Air Lines Inc.   False   367   
...                                                                                         ...   
2024-9 WY Riverton/Lander, WY: Central Wyoming Regional      SkyWest Airlines Inc.  True    376   
          Rock Springs, WY: Southwest Wyoming Regional       SkyWest Airlines Inc.  False    77   
                                                                                    True    238   
          Sheridan, WY: Sheridan County                      SkyWest Airlines Inc.  False   426   
                                                                                    True    403   

                                                                                            6  \
0      1  2                                                  3                      4           
2023-1 AK Adak Island, AK: Adak                              Alaska Airlines Inc.   False   0   
          Anchorage, AK: Ted Stevens Anchorage International Alaska Airlines Inc.   False   0   
                                                                                    True   98   
                                                             American Airlines Inc. True    0   
                                                             Delta Air Lines Inc.   False  71   
...                                                                                        ..   
2024-9 WY Riverton/Lander, WY: Central Wyoming Regional      SkyWest Airlines Inc.  True    5   
          Rock Springs, WY: Southwest Wyoming Regional       SkyWest Airlines Inc.  False   0   
                                                                                    True    0   
          Sheridan, WY: Sheridan County                      SkyWest Airlines Inc.  False   0   
                                                                                    True    0   

                                                                                              7  \
0      1  2                                                  3                      4             
2023-1 AK Adak Island, AK: Adak                              Alaska Airlines Inc.   False    25   
          Anchorage, AK: Ted Stevens Anchorage International Alaska Airlines Inc.   False   781   
                                                                                    True   2131   
                                                             American Airlines Inc. True     25   
                                                             Delta Air Lines Inc.   False   133   
...                                                                                         ...   
2024-9 WY Riverton/Lander, WY: Central Wyoming Regional      SkyWest Airlines Inc.  True     59   
          Rock Springs, WY: Southwest Wyoming Regional       SkyWest Airlines Inc.  False     3   
                                                                                    True     26   
          Sheridan, WY: Sheridan County                      SkyWest Airlines Inc.  False     0   
                                                                                    True     17   

                                                                                            8  \
0      1  2                                                  3                      4           
2023-1 AK Adak Island, AK: Adak                              A

In [30]:
# condensed_origin_df.to_json('assets/origin_summerized.json', orient='table')

In [31]:
calc_dest_df = df[['YR-MN', 'DEST_STATE_ABR','DEST_AIRPORT_NAME','AIRLINE_NAME', 'IS_WEEKDAY']+['congestion', 'time_deviation','speed_deviation','recovery_efficiency']]
calc_dest_df.columns = list(map(str, range(9)))
calc_dest_df = calc_dest_df.groupby(list(map(str, range(5))))[list(map(str, range(5, 9)))].agg(['mean'])
calc_dest_df.columns = calc_dest_df.columns.get_level_values(0)
calc_dest_df

5  \
0      1  2                                                  3                      4                 
2023-1 AK Anchorage, AK: Ted Stevens Anchorage International Alaska Airlines Inc.   False  0.186533   
                                                                                    True   0.168211   
                                                             American Airlines Inc. True   0.062954   
                                                             Delta Air Lines Inc.   False  0.138659   
                                                                                    True   0.105553   
...                                                                                             ...   
2024-9 WY Riverton/Lander, WY: Central Wyoming Regional      SkyWest Airlines Inc.  True   0.306577   
          Rock Springs, WY: Southwest Wyoming Regional       SkyWest Airlines Inc.  False  0.408451   
                                                                                    True   0.318429   
          Sheridan, WY: Sheridan County                      SkyWest Airlines Inc.  False  0.258105   
                                                                                    True   0.335441   

                                                                                                  6  \
0      1  2                                                  3                      4                 
2023-1 AK Anchorage, AK: Ted Stevens Anchorage International Alaska Airlines Inc.   False  0.075723   
                                                                                    True   0.042931   
                                                             American Airlines Inc. True  -0.005195   
                                                             Delta Air Lines Inc.   False  0.057425   
                                                                                    True  -0.016721   
...                                                                                             ...   
2024-9 WY Riverton/Lander, WY: Central Wyoming Regional      SkyWest Airlines Inc.  True  -0.049083   
          Rock Springs, WY: Southwest Wyoming Regional       SkyWest Airlines Inc.  False -0.076923   
                                                                                    True  -0.079519   
          Sheridan, WY: Sheridan County                      SkyWest Airlines Inc.  False -0.084469   
                                                                                    True   0.107009   

                                                                                                  7  \
0      1  2                                                  3                      4                 
2023-1 AK Anchorage, AK: Ted Stevens Anchorage International Alaska Airlines Inc.   False -0.045453   
                                                                                    True  -0.025041   
                                                             American Airlines Inc. True   0.004866   
                                                             Delta Air Lines Inc.   False -0.048422   
                                                                                    True   0.015263   
...                                                                                             ...   
2024-9 WY Riverton/Lander, WY: Central Wyoming Regional      SkyWest Airlines Inc.  True   0.034815   
          Rock Springs, WY: Southwest Wyoming Regional       SkyWest Airlines Inc.  False  0.044118   
                                                                                    True   0.060322   
          Sheridan, WY: Sheridan County                      SkyWest Airlines Inc.  False  0.061132   
                                                                                    True  -0.077589   

                                                                              

In [32]:
condensed_dest_df = df[['YR-MN', 'DEST_STATE_ABR','DEST_AIRPORT_NAME','AIRLINE_NAME', 'IS_WEEKDAY']+DELAYS]
condensed_dest_df.columns = list(map(str, range(10)))
condensed_dest_df = condensed_dest_df.groupby(list(map(str, range(5))))[list(map(str, range(5, 10)))].agg(['sum'])
condensed_dest_df.columns = condensed_dest_df.columns.get_level_values(0)
condensed_dest_df

5  \
0      1  2                                                  3                      4             
2023-1 AK Anchorage, AK: Ted Stevens Anchorage International Alaska Airlines Inc.   False   527   
                                                                                    True   1555   
                                                             American Airlines Inc. True     21   
                                                             Delta Air Lines Inc.   False   212   
                                                                                    True    232   
...                                                                                         ...   
2024-9 WY Riverton/Lander, WY: Central Wyoming Regional      SkyWest Airlines Inc.  True    103   
          Rock Springs, WY: Southwest Wyoming Regional       SkyWest Airlines Inc.  False   107   
                                                                                    True    297   
          Sheridan, WY: Sheridan County                      SkyWest Airlines Inc.  False    53   
                                                                                    True    177   

                                                                                             6  \
0      1  2                                                  3                      4            
2023-1 AK Anchorage, AK: Ted Stevens Anchorage International Alaska Airlines Inc.   False   64   
                                                                                    True   184   
                                                             American Airlines Inc. True     0   
                                                             Delta Air Lines Inc.   False    0   
                                                                                    True     0   
...                                                                                        ...   
2024-9 WY Riverton/Lander, WY: Central Wyoming Regional      SkyWest Airlines Inc.  True    60   
          Rock Springs, WY: Southwest Wyoming Regional       SkyWest Airlines Inc.  False    0   
                                                                                    True     0   
          Sheridan, WY: Sheridan County                      SkyWest Airlines Inc.  False    0   
                                                                                    True     0   

                                                                                              7  \
0      1  2                                                  3                      4             
2023-1 AK Anchorage, AK: Ted Stevens Anchorage International Alaska Airlines Inc.   False   711   
                                                                                    True   1296   
                                                             American Airlines Inc. True      0   
                                                             Delta Air Lines Inc.   False   100   
                                                                                    True     39   
...                                                                                         ...   
2024-9 WY Riverton/Lander, WY: Central Wyoming Regional      SkyWest Airlines Inc.  True      3   
          Rock Springs, WY: Southwest Wyoming Regional       SkyWest Airlines Inc.  False     0   
                                                                                    True      0   
          Sheridan, WY: Sheridan County                      SkyWest Airlines Inc.  False     0   
                                                                                    True     30   

                                                                                            8  \
0      1  2                                                  3                      4           
2023-1 AK Anchorage, AK: Ted Stevens Anchorage In

In [33]:
condensed_dest_df[list(map(str, range(10, 14)))] = calc_dest_df[list(map(str, range(5, 9)))]
condensed_dest_df

5  \
0      1  2                                                  3                      4             
2023-1 AK Anchorage, AK: Ted Stevens Anchorage International Alaska Airlines Inc.   False   527   
                                                                                    True   1555   
                                                             American Airlines Inc. True     21   
                                                             Delta Air Lines Inc.   False   212   
                                                                                    True    232   
...                                                                                         ...   
2024-9 WY Riverton/Lander, WY: Central Wyoming Regional      SkyWest Airlines Inc.  True    103   
          Rock Springs, WY: Southwest Wyoming Regional       SkyWest Airlines Inc.  False   107   
                                                                                    True    297   
          Sheridan, WY: Sheridan County                      SkyWest Airlines Inc.  False    53   
                                                                                    True    177   

                                                                                             6  \
0      1  2                                                  3                      4            
2023-1 AK Anchorage, AK: Ted Stevens Anchorage International Alaska Airlines Inc.   False   64   
                                                                                    True   184   
                                                             American Airlines Inc. True     0   
                                                             Delta Air Lines Inc.   False    0   
                                                                                    True     0   
...                                                                                        ...   
2024-9 WY Riverton/Lander, WY: Central Wyoming Regional      SkyWest Airlines Inc.  True    60   
          Rock Springs, WY: Southwest Wyoming Regional       SkyWest Airlines Inc.  False    0   
                                                                                    True     0   
          Sheridan, WY: Sheridan County                      SkyWest Airlines Inc.  False    0   
                                                                                    True     0   

                                                                                              7  \
0      1  2                                                  3                      4             
2023-1 AK Anchorage, AK: Ted Stevens Anchorage International Alaska Airlines Inc.   False   711   
                                                                                    True   1296   
                                                             American Airlines Inc. True      0   
                                                             Delta Air Lines Inc.   False   100   
                                                                                    True     39   
...                                                                                         ...   
2024-9 WY Riverton/Lander, WY: Central Wyoming Regional      SkyWest Airlines Inc.  True      3   
          Rock Springs, WY: Southwest Wyoming Regional       SkyWest Airlines Inc.  False     0   
                                                                                    True      0   
          Sheridan, WY: Sheridan County                      SkyWest Airlines Inc.  False     0   
                                                                                    True     30   

                                                                                            8  \
0      1  2                                                  3                      4           
2023-1 AK Anchorage, AK: Ted Stevens Anchorage In

In [34]:
# condensed_dest_df.to_json('assets/dest_summerized.json', orient='table')

In [36]:
with open('assets/origin_summerized.json', 'r') as file:
    origin_data = json.load(file)

with open('assets/dest_summerized.json', 'r') as file:
    dest_data = json.load(file)

In [37]:
summerized_data = {"origin": origin_data["data"], "dest": dest_data["data"]}

In [38]:
with open('assets/all_summerized.json', 'w') as file:
    json.dump(summerized_data, file)

In [39]:
with open('assets/all_summerized.json', 'r') as file:
    all_data = json.load(file)

In [40]:
os.remove("assets/origin_summerized.json")
os.remove("assets/dest_summerized.json")

In [41]:
print(len(all_data['origin']), len(origin_data['data']))
print(all_data['origin'] == origin_data["data"])

82174 82174
True


In [42]:
print(len(all_data['dest']), len(dest_data['data']))
print(all_data['dest'] == dest_data["data"])

81973 81973
True


### State Calculations

In [5]:
df['time_slots'] = pd.to_datetime(df[DEPART[0]].astype(str).str.zfill(4), format='%H%M')

In [6]:
def categorize_time(time):
    hour = time.hour
    if 0 <= hour < 5:
        return 0 #'Early Morning'
    elif 5 <= hour < 12:
        return 1 #'Morning'
    elif 12 <= hour < 18:
        return 2 #'Afternoon'
    else:
        return 3 #'Evening'

In [7]:
df['time_slots'] = df['time_slots'].apply(categorize_time)

In [8]:
state_origin_df = df[['ORIGIN_STATE_ABR','ORIGIN_AIRPORT_NAME','time_slots']+DELAYS]
state_origin_df.columns = list(map(str, range(8)))
state_origin_df = state_origin_df.groupby(list(map(str, range(3))))[list(map(str, range(3, 8)))].agg(['sum'])
state_origin_df.columns = state_origin_df.columns.get_level_values(0)
state_origin_df

3      4      5  \
0  1                                                  2                        
AK Adak Island, AK: Adak                              2     62     48    153   
   Anchorage, AK: Ted Stevens Anchorage International 0  20066   2194   8551   
                                                      1  46023  17089  18917   
                                                      2  51432   9593  23404   
                                                      3  67601   1425  21930   
...                                                        ...    ...    ...   
WY Riverton/Lander, WY: Central Wyoming Regional      2   3334    497   1219   
                                                      3      0      0      0   
   Rock Springs, WY: Southwest Wyoming Regional       1   7894    602    767   
                                                      2   2751    490    765   
   Sheridan, WY: Sheridan County                      1   8349   1704    963   

                                                           6      7  
0  1                                                  2              
AK Adak Island, AK: Adak                              2  417   1255  
   Anchorage, AK: Ted Stevens Anchorage International 0  112  13182  
                                                      1    2  21953  
                                                      2  126  60688  
                                                      3  117  54718  
...                                                      ...    ...  
WY Riverton/Lander, WY: Central Wyoming Regional      2    0   2548  
                                                      3    0     46  
   Rock Springs, WY: Southwest Wyoming Regional       1    0   2127  
                                                      2    0   1588  
   Sheridan, WY: Sheridan County                      1    4   4887  

[1054 rows x 5 columns]

In [9]:
# state_origin_df.to_json('assets/state_origin.json', orient='table')

In [10]:
state_dest_df = df[['DEST_STATE_ABR','DEST_AIRPORT_NAME','time_slots']+DELAYS]
state_dest_df.columns = list(map(str, range(8)))
state_dest_df = state_dest_df.groupby(list(map(str, range(3))))[list(map(str, range(3, 8)))].agg(['sum'])
state_dest_df.columns = state_dest_df.columns.get_level_values(0)
state_dest_df

3     4      5  \
0  1                                                  2                       
AK Adak Island, AK: Adak                              1    670    35    170   
                                                      2    175     0      1   
   Anchorage, AK: Ted Stevens Anchorage International 0    796   372    492   
                                                      1  41953  6162  27758   
                                                      2  45186  8763  26139   
...                                                        ...   ...    ...   
WY Rock Springs, WY: Southwest Wyoming Regional       1    796    79    142   
                                                      2    623   437    275   
                                                      3   8985   751    594   
   Sheridan, WY: Sheridan County                      1   3079   202    347   
                                                      3   6140   955    508   

                                                            6      7  
0  1                                                  2               
AK Adak Island, AK: Adak                              1     0    282  
                                                      2     0     14  
   Anchorage, AK: Ted Stevens Anchorage International 0    31   2599  
                                                      1   866  41169  
                                                      2  5463  90451  
...                                                       ...    ...  
WY Rock Springs, WY: Southwest Wyoming Regional       1     0    379  
                                                      2     0   1163  
                                                      3    18   2827  
   Sheridan, WY: Sheridan County                      1     0   1259  
                                                      3     0   5554  

[1122 rows x 5 columns]

In [11]:
# state_dest_df.to_json('assets/state_dest.json', orient='table')

In [12]:
with open('assets/state_origin.json', 'r') as file:
    origin_data = json.load(file)

with open('assets/state_dest.json', 'r') as file:
    dest_data = json.load(file)

In [13]:
state_data = {"origin": origin_data["data"], "dest": dest_data["data"]}

In [14]:
with open('assets/all_state_data.json', 'w') as file:
    json.dump(state_data, file)

In [15]:
with open('assets/all_state_data.json', 'r') as file:
    all_data = json.load(file)

In [16]:
os.remove("assets/state_origin.json")
os.remove("assets/state_dest.json")

In [17]:
print(len(all_data['origin']), len(origin_data['data']))
print(all_data['origin'] == origin_data["data"])


1054 1054
True


In [18]:
print(len(all_data['dest']), len(dest_data['data']))
print(all_data['dest'] == dest_data["data"])

1122 1122
True


### Trend Calculations

In [40]:
df['DATE'] = pd.to_datetime(df[DATES].astype(str).agg('-'.join, axis=1))
df['IS_WEEKDAY'] = pd.to_datetime(df['DATE']).dt.weekday < 5
df['YR-MN'] = df[DATES[:2]].astype(str).agg('-'.join, axis=1)
trend_df = df[['YR-MN','IS_WEEKDAY'] + DELAYS]
trend_df.columns = list(map(str, range(7)))

trend_df = trend_df.groupby(list(map(str, range(2)))).agg(
    col2=('2', lambda x: (x != 0).sum()),
    col3=('3', lambda x: (x != 0).sum()),
    col4=('4', lambda x: (x != 0).sum()),
    col5=('5', lambda x: (x != 0).sum()),
    col6=('6', lambda x: (x != 0).sum()),
    col7=('2', 'sum'),
    col8=('3', 'sum'),
    col9=('4', 'sum'),
    col10=('5', 'sum'),
    col11=('6', 'sum'),
)

trend_df.columns = list(map(str, range(2, 12)))
trend_df

2      3      4    5      6        7       8        9  \
0       1                                                                  
2023-1  False  17926   1600  15023  212  13173   816998  110635   327826   
        True   48996   5449  48997  424  44482  2239344  455288  1496820   
2023-10 False  17759    834  12815  179  15101   800227   54105   273550   
        True   39881   2196  29955  377  34632  1688595  169396   592063   
2023-11 False  14088    773  11899  182  11874   611963   57185   251043   
        True   32746   1821  27945  452  26270  1424486  129232   520815   
2023-12 False  17710   1677  16383  266  15664   777284  161787   370687   
        True   34894   2020  30068  497  29960  1434672  232889   665890   
2023-2  False  15798    793  10839  167  11506   740413   60494   229056   
        True   40102   4357  37566  282  34836  1820531  331944   930465   
2023-3  False  23674   1537  21156  303  21779   973430  106204   569799   
        True   54621   4867  52445  483  49774  2226673  339888  1337707   
2023-4  False  26708   2468  22501  310  24047  1182952  183412   703827   
        True   48994   4378  43130  435  45637  2025004  306990  1122716   
2023-5  False  16213    946  11218  177  12782   721679   60965   260901   
        True   49740   3975  38527  434  43866  2076438  262934   903497   
2023-6  False  23763   2871  18926  257  21578  1142626  199012   518534   
        True   68671   8031  58255  656  66567  3090259  484400  1636151   
2023-7  False  35681   4520  27318  336  34343  1659325  274809   826372   
        True   65326   8240  53750  467  64705  2988506  550338  1629818   
2023-8  False  20483   2148  15047  279  18454   984264  146061   381375   
        True   55947   5841  44360  569  53857  2629381  367348  1164549   
2023-9  False  17541   1569  13714  176  14491   887847  120808   398041   
        True   41886   4231  39035  361  39958  1943850  290059  1146235   
2024-1  False  19718   2764  18604  332  17721   894729  207846   408659   
        True   50648  10635  51961  548  50052  2421632  872066  1410720   
2024-10 False  13393    519   9743   98  10403   582058   39321   204393   
        True   34675   1405  29539  241  28536  1616420  132593   645504   
2024-11 False  15009   1002  12522  138  11634   650466   54768   282886   
        True   32474   3076  33217  169  27680  1413026  250889   946988   
2024-12 False  26013   3002  20083  207  23963  1105831  224431   505614   
        True   48314   5693  44097  357  42545  2119057  510323  1180522   
2024-2  False  14349   1408  14626  168  12611   627406  113473   398802   
        True   30953   2532  31418  388  26024  1355966  190113   783057   
2024-3  False  24434   2271  23692  288  24680  1059570  156081   744214   
        True   41681   3784  42444  493  41402  1807676  228909  1138142   
2024-4  False  15959   1673  14269  190  14666   696291  125822   344105   
        True   44515   4591  44097  445  43976  1975234  291698  1247155   
2024-5  False  19645   2656  19140  183  18896   849914  149780   499223   
        True   67489  10630  68251  525  71277  2948269  807867  2064231   
2024-6  False  29595   3585  23336  321  27562  1352463  258898   650501   
        True   59416   6805  51733  531  58294  2616650  467099  1404996   
2024-7  False  29524   3836  22773  292  28156  1536925  210404   588717   
        True   81630   9042  69339  631  80305  4061501  531575  1905262   
2024-8  False  23834   2827  20588  211  22016  1154710  230312   650059   
        True   56663   7148  55733  453  56281  2616014  457016  1629064   
2024-9  False  15489   1170  13773  111  13337   701966   70330   289464   
        True   33811   3672  34117  215  31187  1539465  214762   776289   

                  10       11  
0       1                      
2023-1  False   4851   677415  
        True   12434  2599003  
2023-10 False   4576   786557  
        True    9174  1722730  
2023-11 False   4292   614

In [43]:
trend_df.to_json('assets/all_trends.json', orient='table')
with open('assets/all_trends.json', 'r') as file:
    trend_data = json.load(file)
trend_data = trend_data["data"]
with open('assets/all_trends.json', 'w') as file:
    json.dump(trend_data, file)